In [15]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
from six.moves.urllib import parse

In [16]:
def get_url(url):
    try:
        with closing(get(url, stream=True)) as response:
            if is_good(response):
                return response.content
            else:
                return None

    except RequestException as e:
        print('Error during requests to {0} : {1}'.format(url, str(e)))
        return None
def is_good(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


In [3]:
showscore = 'https://www.show-score.com/categories/top-scoring-on-broadway'
showscore2 = 'https://www.show-score.com/shows/all?opened=yes'
# no filter
showscore3 = 'https://www.show-score.com/shows/filtered?v2=true'

showscore4_base = 'https://www.show-score.com/shows/filtered.json?v2=true&&page=%'
start4 = [showscore4_base % 1]
page_no = 1
not_end = 

raw_html = get_url(showscore3)
print(len(raw_html))

131144


In [33]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

# browser = webdriver.Chrome()
browser = webdriver.Firefox(executable_path=r'/Users/NainikaDSouza/Desktop/sp19/info4300/cs4300sp2019-jp2254-nmd65-bdg74-ac2582-srm276/geckodriver')
# browser = webdriver.Chrome('/cs4300sp2019-jp2254-nmd65-bdg74-ac2582-srm276/chromedriver')
# browser = webdriver.Firefox()
browser.get('https://www.show-score.com/shows/filtered?v2=true')
# time.sleep(1)
# browser.execute_script("window.scrollTo(0,2000);")
# time.sleep(5)

SCROLL_PAUSE_TIME = 0.5

# Get scroll height
last_height = browser.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)
    
    

    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height



# element = browser.find_element_by_tag_name("body")
# pagedowns = 25
# while pagedowns:
# #     element.send_keys(Keys.PAGE_DOWN)
#     time.sleep(2)
#     browser.execute_script("window.scrollTo(0,1000+100*" + str(pagedowns) + ");")

#     pagedowns -= 1
# show_elements = browser.find_element_by_class_name("show-cell js-show-cell col-sm-6 col-md-4 col-lg-3")
raw_html = browser.page_source

# for show_element in show_elements:
#     print(show_element.text)

In [38]:
import re
html = BeautifulSoup(raw_html, 'html.parser')

# get name
musical_dict = {}
for idx, mess in enumerate(html.select('.show-tile__name')):
#     print(mess)
    messString = str(mess)
    pattern = re.compile('\>(.*?)\n')
    match = pattern.search(messString)
#     if match != None:
    name = match.group(1)
#     print("name: ", name)
    musical_dict[idx] = [name]
#     else:
#         pattern = re.compile('\>(.*?)\.')
#         match = pattern.search(messString)
#         name = match.group(1)
#         print("name: ", name)
    
# get description
for idx,p in enumerate(html.select('p')):
#     print("description: ",p.text)
    if idx in musical_dict:
        musical_dict[idx].append(p.text)
    
# print(musical_dict)
# get score
for idx, mess in enumerate(html.select('.positive.show-tile__score')):
#     print(mess)
    messString = str(mess)
    pattern = re.compile('<span class="positive show-tile__score"\>\n(\d+)')
    match = pattern.search(messString)
    if (match) != None:
#         print(type(match))
        score = match.group(1)
    else:
        score = "No Score"
#     print("score: ", score)
    musical_dict[idx].append(score)
print(musical_dict)

{0: ['Hamilton (NYC)', "Lin-Manuel Miranda's hip-hop infused tuner about Alexander Hamilton began at The Public Theater to rave reviews, then transferred to Broadway where it won eleven 2016 Tony Awards including Best Musical. ", '97'], 1: ["Broadway Sip N' Paint", "Come unwind with us as we un-cork our inner theater geek at Broadway Sip N' Paint.", '95'], 2: ['Wicked (NYC)', 'Now in its fourteenth year on Broadway, this all-ages spectacle follows the untold true story of the Witches of Oz.', '93'], 3: ['Dear Evan Hansen (NYC)', "'Dear Evan Hansen' is about how we share, like, and friend, love, feel, and survive in the modern world. Winner of six 2017 Tony Awards, including Best Musical.", '93'], 4: ['The Lion King (NYC)', "In Julie Taymor's Tony Award winning retelling of Disney's animated hit, puppets and live actors weave a dreamscape of jungle characters, creating the story of young Simba's rise to the throne.", '91'], 5: ['Come From Away (NYC)', 'This Broadway sleeper hit musical 

In [39]:
import csv
with open('showscore.csv','w') as file:
    writer = csv.writer(file, delimiter=',')
    writer.writerow(['Name','Score', 'Description'])
    for idx, musicals in musical_dict.items():
        if len(musicals) > 2:
            writer.writerow([musicals[0],musicals[2], musicals[1]])